In [3]:
import pandas as pd
import numpy as np

In [18]:
with open('corpus') as f:
    corpus = f.read()
    
corpus_list = []
for line in corpus.split('\n'):
    corpus_list.append(line)

label_length = len('__label__2')
y_list = []
X_list = []
for document in corpus_list:
    y_list.append(document[:label_length])
    X_list.append(document[label_length+1:])

processed_y_list = []
for i in y_list:
    if i == '__label__2':
        processed_y_list.append(2)
    else:
        processed_y_list.append(1)
        
y_list = processed_y_list

y_list = np.array(y_list)

y_list = y_list.reshape(-1, 1)

y_list = y_list.ravel()

### Spacy Word2Vec

In [11]:
# loading pretrained model from Spacy


import spacy

import en_core_web_sm

nlp = en_core_web_sm.load()

# use a smaller X for demenstration purpose

Small_X_list = X_list[:500]

word2vec_X = [nlp(sentence).vector for sentence in Small_X_list]


In [25]:
def print_shape(input_list):
    print('the shape is', len(input_list), len(input_list[0]))

In [26]:
print_shape(word2vec_X)

the shape is 500 96


### Manually Added Word Embedding

### FastText

### Glove Embedding

In [157]:
from collections import defaultdict
from nltk import word_tokenize
from nltk.corpus import stopwords

doc = []
for line in open('glove.6B.100d.txt', 'r'):
    doc.append(line.replace('\n', ''))

glove_dict = defaultdict(list)
stopwords = stopwords.words('english')

for line in doc:
    word = line.split(' ')[0]
    vector = np.asarray(line.split(' ')[1:], dtype='float32')
    glove_dict[word] = vector

In [179]:
def sent2vec(sentence):
    
    sentece_words = str(sentence).lower()
    sentece_words = word_tokenize(sentece_words)
    sentece_words = [word for word in sentece_words if not word in stopwords]
    sentece_words = [word for word in sentece_words if word.isalpha()]
    
    sentence_word_vectors = []
    
    for word in sentece_words:
        sentence_word_vectors.append(glove_dict[word])
            
    sentence_word_vectors = np.array(sentence_word_vectors)
    
    sum_sentence_word_vectors = sentence_word_vectors.sum(axis=0)
    
    if type(sum_sentence_word_vectors) != np.ndarray:
        
        return np.zeros(100)
    
    # 100 is because the data we are using is 100 dimension
    
    normalized_sentence_word_vectors = sum_sentence_word_vectors / len(sentence_word_vectors)
    
    return normalized_sentence_word_vectors


In [180]:
sent2vec(X_train[0])

array([-5.61145954e-02,  3.51955324e-01,  3.86061639e-01, -1.97051018e-01,
       -1.72846004e-01,  1.36576906e-01, -1.90544561e-01,  1.33789271e-01,
       -1.67788919e-02, -2.71716565e-01,  1.92401558e-01,  3.52062695e-02,
        8.22529644e-02, -1.34434909e-01,  6.45520389e-02, -1.62273034e-01,
       -6.15987182e-02,  1.54483184e-01, -1.76681414e-01,  4.08039272e-01,
        1.99222639e-01,  1.15231499e-01, -5.48733585e-02, -2.06511334e-01,
        7.31869563e-02,  3.01482566e-02, -3.14367265e-01, -4.66888577e-01,
        2.48672850e-02, -1.58738300e-01, -2.18046710e-01,  4.18308973e-01,
       -9.43074841e-03, -1.35022312e-01,  9.99217033e-02,  2.45048061e-01,
       -2.90057212e-01, -2.31278595e-02,  3.08087881e-04, -9.92591456e-02,
       -2.61613190e-01, -9.07974690e-02,  5.52531220e-02, -4.26564395e-01,
       -3.07398975e-01, -1.02058269e-01,  5.85928224e-02, -3.47306103e-01,
        1.82511553e-01, -9.18483913e-01,  3.07927072e-01, -1.86705321e-01,
       -2.84779239e-02,  

### TFIDF Word Embedding

In [19]:
# adjusted weights based on corpus

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(X_list, test_size = 0.2)

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(X_train)

X_train_vectorized = tfidf_vectorizer.transform(X_train)
X_test_vectorized = tfidf_vectorizer.transform(X_test)

In [28]:
X_train_vectorized

<8000x28234 sparse matrix of type '<class 'numpy.float64'>'
	with 443569 stored elements in Compressed Sparse Row format>

### Bag of Words Embedding

In [30]:
# does not adjust weights. just count the frequencies

from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
count_vectorizer.fit(X_train)

X_train_vectorized = tfidf_vectorizer.transform(X_train)
X_test_vectorized = tfidf_vectorizer.transform(X_test)

In [31]:
X_train_vectorized

<8000x28234 sparse matrix of type '<class 'numpy.float64'>'
	with 443569 stored elements in Compressed Sparse Row format>

### Keras Embedding

In [32]:
# This embedding purely just encode each word to a number. does not involve any calculation or representation

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# first select the 10000 most unique common words
max_features = 10000

keras_tokenizer = Tokenizer(num_words=max_features)
keras_tokenizer.fit_on_texts(X_train)

X_train_vectorized_keras = keras_tokenizer.texts_to_sequences(X_train)

X_test_vectorized_keras = keras_tokenizer.texts_to_sequences(X_test)



In [37]:
# to make sure each observation has the same length:
# let's assume the maximum word is 500
max_word = 500

X_train_sequence = pad_sequences(X_train_vectorized_keras, maxlen = max_word)

X_test_sequence = pad_sequences(X_test_vectorized_keras, maxlen = max_word)

In [38]:
print_shape(X_train_sequence)

the shape is 8000 500
